## This Notebook contains code to pull up list of files from Google Drive
####            a)  Pull up list of documents in a Google Drive account mapped to the service account referenced below by private key
####            b)  Pull up the contents of the Google sheets and render it into a Dataframe 
####            c)  Display and store using %store magic function to access across notebooks


In [52]:
# Import libraries 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from apiclient.discovery import build #Util for API calls
import gspread # For connection to google sheets
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g # d2g will be used once ready to upload data back to sheets

In [53]:
# Configure the connection 
scopes = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive.readonly']
key_file_location = 'gsheetsprivkey\serviceaccount.json'

# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name(key_file_location,scopes)

# Authorise your Jupyter Notebook to connect to Google Drive API using private key credentials in 'credentials'
gc = gspread.authorize(credentials)


# Function to be able to get service for google drive
def get_service(api_name, api_version, scopes=scopes, key_file_location=key_file_location):
    """Get a service that communicates to a Google API.

    Args:
        api_name: The name of the api to connect to.
        api_version: The api version to connect to.
        scopes: A list auth scopes to authorize for the application.
        key_file_location: The path to a valid service account JSON key file.

    Returns:
        A service that is connected to the specified API.
    """

    # Build the service object.
    service = build(api_name, api_version, credentials=credentials)

    return service

# Retrieve list of files in Google Drive
# Interested only in spreadsheets within folder, hence using query as arg option within list function
itemsInDrive = get_service("drive","v3").files().list(q="mimeType='application/vnd.google-apps.spreadsheet'").execute()

#Retrieving value of 'files' key which has details of all the files we are interested in.
filesInDrive = itemsInDrive['files']



In [54]:
# Converting results to Dictionary with 'name' as the index of each entry.
# This allows for easier reading of the data structure using the name
filesInDriveDict = {};

for file in filesInDrive:
    itemList = [item for item in file.items() if item[0] != 'name' ] #Strip out name 
    filesInDriveDict[file['name']] = dict(itemList) # Build dict with value of 'name' as key
    
#Optional, if you need this to be shared across Notebooks.
#In my case I needed list of filesInDriveDict in the drive to be available across other Notebooks
%store filesInDriveDict

Stored 'filesInDriveDict' (dict)


In [55]:
# Helper function that retrieves the id from Google Drive to pass to Google Sheets API
# season naming format is EPL_YYYY_YY , E.g. EPL_2004_05

def get_fileID_from_name(seasonName,filesInDriveDict):
    if seasonName in filesInDriveDict:
        return filesInDriveDict[seasonName]['id']
    raise Exception("File name passed to get_fileID_from_name is invalid") #If invalid name passed


In [56]:
# Pass name of the file to function defined above
spreadsheet_key = get_fileID_from_name("EPL_1993_94",filesInDriveDict)
#spreadsheet_key is the internal key stored in Google drive for that file
workbook = gc.open_by_key(spreadsheet_key)
#By default load 1st sheet of the wor , i.e. index = 0
sheet = workbook.get_worksheet(0)
#get_all_values returns list of lists with first list as column headers
values = sheet.get_all_values()
# Pulling the data and transform it to the data frame .1st row is header , remaining are actual values
pd_data = pd.DataFrame(values[1:], columns = values[0])
#converting string format dates to datetime object
pd_data['Date'] = pd.to_datetime(pd_data['Date'])
pd_data



,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,E0,1993-08-14,Arsenal,Coventry,0,3,A
1,E0,1993-08-14,Aston Villa,QPR,4,1,H
2,E0,1993-08-14,Chelsea,Blackburn,1,2,A
3,E0,1993-08-14,Liverpool,Sheffield Weds,2,0,H
4,E0,1993-08-14,Man City,Leeds,1,1,D
...,...,...,...,...,...,...,...
457,E0,1994-07-05,Sheffield Weds,Man City,1,1,D
458,E0,1994-07-05,Swindon,Leeds,0,5,A
459,E0,1994-07-05,Tottenham,QPR,1,2,A
460,E0,1994-07-05,West Ham,Southampton,3,3,D


In [57]:
#Optional, using magic function if you want to pull up data via one call to the API and persist across notebooks

%store pd_data

Stored 'pd_data' (DataFrame)
